In [ ]:
# 모듈 임포트
import time
import board
import busio
import adafruit_ads1x15.ads1115 as ADS
from adafruit_ads1x15.analog_in import AnalogIn
import joblib
import numpy as np
import psutil
import pandas as pd
from sklearn.preprocessing import StandardScaler
import csv
from datetime import datetime

In [ ]:
# 모델 및 스케일러 로드
model = joblib.load('./model/one_class_svm_model.joblib')
scaler = joblib.load('./model/scaler.joblib')

In [ ]:
# I2C 및 ADC 설정
i2c = busio.I2C(board.SCL, board.SDA)
ads = ADS.ADS1115(i2c) # (아날로그 → 디지털 변환기) 객체
ads.gain = 1 # 증폭 배율 설정

# ADS1115 채널 선택
voltage_ch = AnalogIn(ads, ADS.P0)
current_ch = AnalogIn(ads, ADS.P1)

In [ ]:
# 센서 기준 오프셋
ZMPT_offset = 2.554
ZMPT_scale = 1000.0     # 실험을 통해 보정 예정

ACS712_offset = 2.508
ACS712_sensitivity = 0.1  # 또는 0.05

In [ ]:
# 파생 피처 생성 함수
def add_features(voltage_seq, current_seq):
    voltage = np.mean(voltage_seq)
    current = np.mean(current_seq)
    voltage_diff = np.abs(voltage_seq[-1] - voltage_seq[-2])
    current_diff = np.abs(current_seq[-1] - current_seq[-2])
    voltage_ma = np.mean(voltage_seq[-5:])
    current_ma = np.mean(current_seq[-5:])
    power = voltage * current
    power_diff = np.abs(power - (voltage_seq[-2] * current_seq[-2]))
    return [voltage, current, voltage_diff, current_diff, voltage_ma, current_ma, power, power_diff]

In [ ]:
def postprocess_anomalies_realtime(anomaly_buffer, min_consecutive=8):
    if len(anomaly_buffer) < min_consecutive:
        return False
    
    # 정확히 연속된 1이 min_consecutive 이상인지 확인
    count = 0
    for val in reversed(anomaly_buffer): # 최근 값부터 역순으로 확인
        if val == 1:
            count += 1
        else:
            break
    
    return count >= min_consecutive

In [52]:
# 데이터 파일 로드 (테스트 시 사용)
column_names = ['timestamp', 'v_raw', 'c_raw', 'voltage', 'current']
df = pd.read_csv('./진짜 마지막.csv', names=column_names, header=None)
df = df.dropna(subset=['voltage', 'current']).reset_index(drop=True)
print(df.describe())
csv_filename = "진짜 마지막2.csv"
# 실시간 센서 읽기 및 예측
try:
    print("🚀 실시간 아크 감지 시작합니다!")

    data_buffer = []
    anomaly_buffer = []
    ANOMALY_THRESHOLD = 4
    first_anomaly_detected = False
    
    start_time = time.perf_counter()
    anomaly_start_time = None  # 최초의 이상 신호가 들어온 시간
    idx = -1
    
    while True:
        idx += 1
        # 테스트 데이터 이용 (보정X)
        # voltage_data = df['voltage'][idx]
        # current_data = df['current'][idx]
        voltage_data = df['voltage'].iloc[idx]
        current_data = df['current'].iloc[idx]
        

        # 💡 필요에 따라 아래를 전환하세요
        USE_SENSOR = False  # 센서 사용 시 True, 데이터 사용 시 False

        if USE_SENSOR:
            try:
                # 센서 입력 시도
                v_raw_sensor = voltage_ch.voltage 
                c_raw_sensor = current_ch.voltage

                voltage = (v_raw_sensor - ZMPT_offset) * ZMPT_scale
                current = (c_raw_sensor - ACS712_offset) / ACS712_sensitivity

                prev_voltage = voltage
                prev_current = current

            except (OSError, ValueError, RuntimeError) as e:
                print(f"⚠️ 센서 오류 발생: {e}")
                print("🩹 이전 값을 사용하여 계속 진행합니다.")
                voltage = 0
                current = 0
        else:
            voltage, current = voltage_data, current_data

        data_buffer.append((voltage, current))
        # CSV에 현재 데이터 저장
        with open(csv_filename, mode='a', newline='') as file:
            writer = csv.writer(file)
            timestamp = datetime.now().isoformat()
            writer.writerow([timestamp, voltage, current, anomaly if 'anomaly' in locals() else 0])

        if len(data_buffer) >= 8:
            voltage_seq, current_seq = zip(*data_buffer[-8:])
            features = add_features(voltage_seq, current_seq)
            features_scaled = scaler.transform(np.array(features).reshape(1, -1))
            y_pred = model.predict(features_scaled)
            anomaly = int(y_pred.flatten()[0])

            anomaly_buffer.append(anomaly)
            if len(anomaly_buffer) > ANOMALY_THRESHOLD:
                anomaly_buffer.pop(0)

            # 이상 신호 시간 체크 로직
            if anomaly == 1:
                if anomaly_start_time is None:  
                    anomaly_start_time = time.perf_counter()
            else:
                anomaly_start_time = None

            # 후처리 적용 후 8개 연속 이상 탐지 시
            if postprocess_anomalies_realtime(anomaly_buffer, min_consecutive=ANOMALY_THRESHOLD):
                print("⚡ 아크 이상 감지!", idx)
                if not first_anomaly_detected and anomaly_start_time is not None:
                    total_elapsed = time.perf_counter() - start_time
                    actual_elapsed = time.perf_counter() - anomaly_start_time
                    print(f"⏱️ 전체 소요 시간: {total_elapsed:.6f}초")
                    print(f"⏱️ 연속 8개 이상 신호까지 실제 소요 시간: {actual_elapsed:.6f}초")
                    first_anomaly_detected = True
                    break

        cpu_usage = psutil.cpu_percent(interval=0.0)
        memory_usage = psutil.Process().memory_info().rss / 1024 ** 2
        print(f"🧠 메모리 사용량: {memory_usage:.2f} MB, 🧮 CPU 사용률: {cpu_usage:.2f}% , idx: {idx}")

        time.sleep(0.0001)

except KeyboardInterrupt:
    print("🛑 실시간 감지를 종료합니다.")

       v_raw  c_raw  voltage  current
count    0.0    0.0      0.0      0.0
mean     NaN    NaN      NaN      NaN
std      NaN    NaN      NaN      NaN
min      NaN    NaN      NaN      NaN
25%      NaN    NaN      NaN      NaN
50%      NaN    NaN      NaN      NaN
75%      NaN    NaN      NaN      NaN
max      NaN    NaN      NaN      NaN
🚀 실시간 아크 감지 시작합니다!


IndexError: single positional indexer is out-of-bounds